# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook and serves as an extension of the API documentation to show what the results will look like depending on what inputs are provided.

<div class="alert alert-block alert-warning">
<b>Note:</b>
As of v0.5, most outputs, including availability, return values as a pandas DataFrame, regardless of the aggregation level.
</div>


In [1]:
from pprint import pprint

import pandas as pd

from wombat.core import Simulation, Metrics

# Clean up the aesthetics for the pandas outputs
pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 30)
pd.set_option("display.max_columns", 10)

## Table of Contents

Below is a list of top-level sections to demonstrate how to use WOMBAT's `Metrics` class methods and an explanation of each individual metric.
 - [Setup](#setup): Running a simulation to gather the results
 - [Availability](#availability): Time-based and energy-based availability
 - [Capacity Factor](#capacity-factor): Gross and net capacity factor
 - [Task Completion Rate](#task-completion-rate): Task completion metrics
 - [Equipment Costs](#equipment-costs): Cost breakdowns by servicing equipment
 - [Service Equipment Utilization Rate](#service-equipment-utilization-rate): Utilization of servicing equipment
 - [Vessel-Crew Hours at Sea](#vessel-crew-hours-at-sea): Number of crew or vessel hours spent at sea
 - [Number of Tows](#number-of-tows): Number of tows breakdowns
 - [Labor Costs](#labor-costs): Breakdown of labor costs
 - [Equipment and Labor Costs](#equipment-and-labor-costs): Combined servicing equipment and labor cost breakdown
 - [Component Costs](#component-costs): Materials costs
 - [Fixed Cost Impacts](#fixed-cost-impacts): Total fixed costs
 - [OpEx](#opex): Project OpEx
 - [Process Times](#process-times): Timing of various stages of repair and maintenance
 - [Power Production](#power-production): Potential and actually produced power
 - [Net Present Value](#net-present-value): Project NPV calculator
 - [PySAM-Powered Results](#pysam-powered-results): PySAM results, if configuration is provided

<a id="setup"></a>
## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

To simplify this process, a feature has been added to save the simulation outputs required to generate the Metrics inputs and a method to reload those outputs as inputs.

In [2]:
sim = Simulation("DINWOODIE", "base.yaml")

# Both of these parameters are True by default for convenience
sim.run(create_metrics=True, save_metrics_inputs=True)

# Load the metrics data
fpath = sim.env.metrics_input_fname.parent
fname = sim.env.metrics_input_fname.name
metrics = Metrics.from_simulation_outputs(fpath, fname)

# Delete the log files now that they're loaded in
sim.env.cleanup_log_files(log_only=False)

# Alternatively, in this case because the simulation was run, we can use the
# following for convenience convenience only
metrics = sim.metrics

<a id="availability"></a>
## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [3]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm").values[0][0]
print(f"Project total: {total * 100:.1f}%")

Project total: 94.0%


In [4]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,...,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.97,0.98,0.98,0.95,0.88,...,0.93,0.92,0.96,0.92,0.97


In [5]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,windfarm
year,
2003,0.98
2004,0.91
2005,0.91
2006,0.95
2007,0.95
2008,0.95
2009,0.95
2010,0.94
2011,0.94


In [6]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,windfarm
month,
1,0.95
2,0.93
3,0.93
4,0.92
5,0.93
6,0.93
7,0.95
8,0.95
9,0.95


In [7]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm")

windfarm
year month          
2003 1          0.99
     2          0.99
     3          0.99
     4          0.99
     5          0.99
...              ...
2012 8          0.93
     9          0.93
     10         0.93
     11         0.96
     12         0.95

[120 rows x 1 columns]

<a id="capacity-factor"></a>
## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [8]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm").values[0][0]
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm").values[0][0]
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.45%
Gross Capacity Factor: 0.48%


In [9]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,...,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.46,0.47,0.47,0.45,0.42,...,0.45,0.43,0.46,0.44,0.46


In [10]:
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

,windfarm
year,
2003,0.41
2004,0.42
2005,0.45
2006,0.44
2007,0.48
2008,0.49
2009,0.45
2010,0.40
2011,0.47


In [11]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

,windfarm
month,
1,0.59
2,0.49
3,0.49
4,0.40
5,0.37
6,0.31
7,0.32
8,0.35
9,0.45


In [12]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm")

windfarm
year month          
2003 1          0.62
     2          0.37
     3          0.38
     4          0.53
     5          0.35
...              ...
2012 8          0.30
     9          0.52
     10         0.50
     11         0.52
     12         0.59

[120 rows x 1 columns]

<a id="task-completion-rate"></a>
## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [13]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project").values[0][0]
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project").values[0][0]
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project").values[0][0]
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 98%
Unscheduled Task Completion Rate: 100%
    Overall Task Completion Rate: 100%


In [14]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

,Completion Rate
year,
2003,1.00
2004,0.96
2005,1.03
2006,1.00
2007,0.99
2008,1.00
2009,0.99
2010,1.00
2011,0.98


In [15]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

,Completion Rate
month,
1,0.76
2,0.87
3,0.90
4,1.17
5,1.07
6,1.11
7,1.09
8,1.07
9,1.09


In [16]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year")

Completion Rate
year month                 
2003 1                 0.93
     2                 1.06
     3                 0.93
     4                 1.08
     5                 1.00
...                     ...
2012 8                 1.30
     9                 0.97
     10                1.12
     11                1.00
     12                1.00

[120 rows x 1 columns]

<a id="equipment-costs"></a>
## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [17]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total.values[0][0] / metrics.project_capacity:,.2f}/MW")

Project total: $370,258.91/MW


In [18]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"6,095,377.61","6,094,028.65","6,113,752.61","1,923,497.88","68,635,482.64"


In [19]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,equipment_cost
year,
2003,"1,851,736.98"
2004,"11,303,909.21"
2005,"12,648,693.58"
2006,"11,444,696.07"
2007,"5,620,180.77"
2008,"6,409,965.97"
2009,"6,621,747.82"
2010,"11,703,130.72"
2011,"7,634,128.99"


In [20]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
month,,,,,
1,"503,434.90","506,278.65","505,111.98",0.00,"5,487,687.50"
2,"460,596.35","463,020.83","462,528.65",0.00,"5,367,187.50"
3,"510,289.06","509,833.33","510,872.40","258,345.13","4,048,559.03"
4,"511,328.13","511,255.21","512,111.98","244,812.14","5,990,343.75"
5,"528,846.35","526,841.15","529,174.48","19,000.00","3,565,265.62"
6,"508,265.63","508,575.52","509,195.31","201,539.75","11,575,977.43"
7,"530,687.50","530,687.50","531,216.15","273,479.27","6,404,687.50"
8,"532,200.52","532,601.56","533,658.85","308,156.25",0.00
9,"497,802.08","499,315.10","498,203.13","203,333.76","5,000,914.93"


In [21]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True)

Crew Transfer Vessel 1  Crew Transfer Vessel 2  \
year month                                                   
2003 1                   50,385.42               52,864.58   
     2                   47,250.00               47,250.00   
     3                   53,375.00               53,375.00   
     4                   50,622.40               50,622.40   
     5                   54,250.00               52,645.83   
...                            ...                     ...   
2012 8                   53,885.42               54,104.17   
     9                   47,231.77               49,401.04   
     10                  51,096.35               51,260.42   
     11                  52,500.00               52,500.00   
     12                  53,958.33               52,664.06   

            Crew Transfer Vessel 3  Field Support Vessel  Heavy Lift Vessel  
year month                                                                   
2003 1                   51,260.42                  0.00               0.00  
     2                   47,250.00                  0.00               0.00  
     3                   53,375.00                  0.00               0.00  
     4                   51,406.25                  0.00               0.00  
     5                   54,250.00                  0.00               0.00  
...                            ...                   ...                ...  
2012 8                   53,885.42                  0.00               0.00  
     9                   48,799.48                  0.00         538,414.93  
     10                  51,242.19            195,793.02       3,097,522.57  
     11                  51,625.00             65,312.50       1,390,625.00  
     12                  52,919.27                  0.00               0.00  

[120 rows x 5 columns]

<a id="service-equipment-utilization-rate"></a>
## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [22]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,1.00,1.00,1.00,0.92,0.96


In [23]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,1.00,1.00,1.00,0.00,0.00
2004,1.00,1.00,1.00,0.86,0.96
2005,1.00,1.00,1.00,0.00,0.96
2006,1.00,1.00,1.00,0.94,0.95
2007,1.00,1.00,1.00,0.93,0.91
2008,1.00,1.00,1.00,0.94,0.96
2009,1.00,1.00,1.00,0.94,0.94
2010,1.00,1.00,1.00,0.93,0.98
2011,1.00,1.00,1.00,0.94,0.94


<a id="vessel-crew-hours-at-sea"></a>
## Vessel-Crew Hours at Sea

The number of vessel hours or crew hours at sea for offshore wind power plant simulations.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed for each vessel simulated
 - `False`: computed only as a sum of all vessels
 
`vessel_crew_assumption`: A dictionary of vessel names (`ServiceEquipment.settings.name`, but also found at `Metrics.service_equipment_names`) and the number of crew onboard at any given time. The application of this assumption transforms the results from vessel hours at sea to crew hours at sea.

In [24]:
# Project total, not broken out by vessel
metrics.vessel_crew_hours_at_sea(frequency="project", by_equipment=False)

,Total Crew Hours at Sea
0,"125,619.04"


In [25]:
# Annual project totals, broken out by vessel
metrics.vessel_crew_hours_at_sea(frequency="annual", by_equipment=True)

,year,Total Crew Hours at Sea,Crew Transfer Vessel 1,Crew Transfer Vessel 3,Crew Transfer Vessel 2,Heavy Lift Vessel,Field Support Vessel
0,2003,"4,913.00","1,629.50","1,592.00","1,691.50",0.00,0.00
1,2004,"14,321.57","3,112.00","3,128.75","3,238.50","3,733.32","1,109.00"
2,2005,"13,059.98","3,125.75","3,069.75","3,102.75","3,761.73",0.00
3,2006,"13,858.00","3,278.50","3,146.75","3,153.00","3,680.75",599.00
4,2007,"12,609.50","3,508.00","3,425.25","3,399.25","1,644.00",633.00
5,2008,"12,347.97","3,213.25","3,264.50","3,366.25","1,879.47",624.50
6,2009,"12,606.00","3,429.50","3,319.25","3,464.00","1,777.75",615.50
7,2010,"13,723.26","3,092.50","3,119.75","3,183.50","3,727.51",600.00
8,2011,"14,012.50","3,492.00","3,395.50","3,338.50","3,167.50",619.00
9,2012,"14,167.25","3,353.75","3,123.00","3,169.25","3,882.25",639.00


<a id="number-of-tows"></a>
## Number of Tows

The number of tows metric will only produce results if any towing actually occurred, otherwise a dataframe with a value of 0 is returned, as is the case in this demonstration.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_tug` options:
 - `True`: computed for each tugboat (towing vessel)
 - `False`: computed only as a sum of all tugboats
 
`by_direction` options:
 - `True`: computed for each direction (to port or to site)
 - `False`: computed as a sum for both directions

In [26]:
# Project Total
# NOTE: This example has no towing, so it will return 0
metrics.number_of_tows(frequency="project")

,total_tows
0,0


<a id="labor-costs"></a>
## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [27]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total.values[0][0] / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [28]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [29]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,total_labor_cost
year,
2003,0.00
2004,0.00
2005,0.00
2006,0.00
2007,0.00
2008,0.00
2009,0.00
2010,0.00
2011,0.00


In [30]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,total_labor_cost
month,
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [31]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False)

total_labor_cost
year month                  
2003 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
...                      ...
2012 8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00

[120 rows x 1 columns]

<a id="equipment-and-labor-costs"></a>
## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [32]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,total_cost
reason,
Maintenance,"2,919,601.56"
Repair,"18,756,115.45"
Crew Transfer,"1,116,171.88"
Site Travel,0.00
Mobilization,"7,500,000.00"
Weather Delay,"30,732,941.84"
No Requests,"7,438,922.74"
Not in Shift,"20,398,385.90"


In [33]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
reason,,,,,
Maintenance,0.00,0.00,0.00,"2,919,601.56","2,919,601.56"
Repair,0.00,0.00,0.00,"18,756,115.45","18,756,115.45"
Crew Transfer,0.00,0.00,0.00,"1,116,171.88","1,116,171.88"
Site Travel,0.00,0.00,0.00,0.00,0.00
Mobilization,0.00,0.00,0.00,"7,500,000.00","7,500,000.00"
Weather Delay,0.00,0.00,0.00,"30,732,941.84","30,732,941.84"
No Requests,0.00,0.00,0.00,"7,438,922.74","7,438,922.74"
Not in Shift,0.00,0.00,0.00,"20,398,385.90","20,398,385.90"


In [34]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False)

total_cost
year reason                    
2003 Maintenance       4,375.00
     Repair          279,653.65
     Crew Transfer    32,666.67
     Site Travel           0.00
     Mobilization          0.00
...                         ...
2012 Site Travel           0.00
     Mobilization  1,000,000.00
     Weather Delay 6,123,945.31
     No Requests     677,247.40
     Not in Shift  2,630,347.71

[80 rows x 1 columns]

In [35]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False)

total_cost
month reason                    
1     Maintenance     120,658.85
      Repair        1,025,365.45
      Crew Transfer    72,158.85
      Site Travel           0.00
      Mobilization  1,000,000.00
...                          ...
12    Site Travel           0.00
      Mobilization    500,000.00
      Weather Delay 1,993,648.44
      No Requests     765,989.58
      Not in Shift    224,364.58

[96 rows x 1 columns]

In [36]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False)

total_cost
year month reason                   
2003 1     Maintenance          0.00
           Repair          28,747.40
           Crew Transfer    3,171.88
           Site Travel          0.00
           Mobilization         0.00
...                              ...
2012 12    Site Travel          0.00
           Mobilization         0.00
           Weather Delay    3,208.33
           No Requests    103,614.58
           Not in Shift    19,541.67

[960 rows x 1 columns]

<a id="component-costs"></a>
## Component Costs

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [37]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,total_cost
component,
turbine,"111,214,652.15"


In [38]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

materials_cost  total_labor_cost  equipment_cost  \
component action                                                          
turbine   delay                  0.00              0.00   50,981,763.25   
          maintenance            0.00              0.00    2,919,601.56   
          repair                 0.00              0.00   18,756,115.45   

                         total_cost  
component action                     
turbine   delay       50,981,763.25  
          maintenance  2,919,601.56  
          repair      18,756,115.45

In [39]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
year,component,,,,
2003,turbine,"628,500.00",0.00,"536,466.15","1,164,966.15"
2004,turbine,"3,407,500.00",0.00,"9,822,498.26","13,229,998.26"
2005,turbine,"5,225,500.00",0.00,"11,167,881.08","16,393,381.08"
2006,turbine,"4,593,000.00",0.00,"8,791,268.12","13,384,268.12"
2007,turbine,"2,688,500.00",0.00,"4,659,651.04","7,348,151.04"
2008,turbine,"3,895,500.00",0.00,"5,296,156.07","9,191,656.07"
2009,turbine,"3,720,000.00",0.00,"5,480,213.54","9,200,213.54"
2010,turbine,"5,151,500.00",0.00,"9,994,373.26","15,145,873.26"
2011,turbine,"3,296,000.00",0.00,"6,092,988.36","9,388,988.36"


In [40]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
month,component,,,,
1,turbine,"1,836,000.00",0.00,"5,379,421.88","7,215,421.88"
2,turbine,"2,684,000.00",0.00,"6,410,533.85","9,094,533.85"
3,turbine,"2,615,500.00",0.00,"4,975,878.12","7,591,378.12"
4,turbine,"3,784,000.00",0.00,"6,489,520.83","10,273,520.83"
5,turbine,"4,388,000.00",0.00,"4,503,565.11","8,891,565.11"
6,turbine,"5,325,500.00",0.00,"11,567,267.18","16,892,767.18"
7,turbine,"5,920,000.00",0.00,"7,062,492.95","12,982,492.95"
8,turbine,"2,234,000.00",0.00,"1,295,947.92","3,529,947.92"
9,turbine,"2,037,500.00",0.00,"4,332,865.45","6,370,365.45"


In [41]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost  \
year month component                                                     
2003 1     turbine         94,000.00              0.00       61,213.54   
     2     turbine         92,000.00              0.00       57,968.75   
     3     turbine         49,500.00              0.00       40,796.88   
     4     turbine         39,500.00              0.00       50,020.83   
     5     turbine         55,000.00              0.00       33,687.50   
     6     turbine         59,000.00              0.00       40,486.98   
     7     turbine         18,000.00              0.00       24,098.96   
     8     turbine         36,500.00              0.00       37,442.71   
     9     turbine         74,500.00              0.00       53,958.33   
     10    turbine         36,500.00              0.00       46,484.38   
     11    turbine         54,000.00              0.00       38,080.73   
     12    turbine         20,000.00              0.00       52,226.56   
2004 1     turbine        149,500.00              0.00      144,375.00   
     2     turbine        161,000.00              0.00      129,664.06   
     3     turbine        537,500.00              0.00    2,978,840.28   
     4     turbine        626,000.00              0.00      610,315.97   
     5     turbine        242,000.00              0.00      155,039.06   
     6     turbine        222,500.00              0.00      150,828.13   
     7     turbine        259,500.00              0.00      157,500.00   
     8     turbine        110,500.00              0.00       82,213.54   
     9     turbine         39,500.00              0.00       49,947.92   
     10    turbine        116,000.00              0.00    2,461,359.73   
     11    turbine        847,500.00              0.00    2,824,138.53   
     12    turbine         96,000.00              0.00       78,276.04   
2005 1     turbine        146,500.00              0.00      130,739.58   
     2     turbine         85,000.00              0.00      124,140.63   
     3     turbine        177,500.00              0.00      152,414.06   
     4     turbine        258,500.00              0.00      152,250.00   

                       total_cost  
year month component               
2003 1     turbine     155,213.54  
     2     turbine     149,968.75  
     3     turbine      90,296.88  
     4     turbine      89,520.83  
     5     turbine      88,687.50  
     6     turbine      99,486.98  
     7     turbine      42,098.96  
     8     turbine      73,942.71  
     9     turbine     128,458.33  
     10    turbine      82,984.38  
     11    turbine      92,080.73  
     12    turbine      72,226.56  
2004 1     turbine     293,875.00  
     2     turbine     290,664.06  
     3     turbine   3,516,340.28  
     4     turbine   1,236,315.97  
     5     turbine     397,039.06  
     6     turbine     373,328.13  
     7     turbine     417,000.00  
     8     turbine     192,713.54  
     9     turbine      89,447.92  
     10    turbine   2,577,359.73  
     11    turbine   3,671,638.53  
     12    turbine     174,276.04  
2005 1     turbine     277,239.58  
     2     turbine     209,140.63  
     3     turbine     329,914.06  
     4     turbine     410,750.00

<a id="fixed-cost-impacts"></a>
## Fixed Cost Impacts

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [42]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [43]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"16,013,230.75"


In [44]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"16,013,230.75"


In [45]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,...,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,"16,013,230.75"


In [46]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
year,,,,,,,
2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2004,0.00,0.00,0.00,0.00,0.00,0.00,"1,604,391.58"
2005,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2006,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2007,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2008,0.00,0.00,0.00,0.00,0.00,0.00,"1,604,391.58"
2009,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2010,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2011,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


In [47]:
# Project annualized totals at the low level by month
metrics.project_fixed_costs(frequency="annual", resolution="low")

,operations
year,
2003,"1,600,008.00"
2004,"1,604,391.58"
2005,"1,600,008.00"
2006,"1,600,008.00"
2007,"1,600,008.00"
2008,"1,604,391.58"
2009,"1,600,008.00"
2010,"1,600,008.00"
2011,"1,600,008.00"


In [48]:
# Project annualized totals at the low level by year and month
metrics.project_fixed_costs(frequency="month-year", resolution="low")

operations
year month            
2003 1      135,891.09
     2      122,740.34
     3      135,891.09
     4      131,507.51
     5      135,891.09
...                ...
2012 8      135,891.09
     9      131,507.51
     10     135,891.09
     11     131,507.51
     12     135,891.09

[120 rows x 1 columns]

<a id="opex"></a>
## OpEx

Here, we will go through the various input definitions to get the project OpEx

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [49]:
metrics.opex("project")

,OpEx
0,"142,316,370.13"


In [50]:
metrics.opex("annual")

,OpEx
year,
2003,"4,080,244.98"
2004,"16,315,800.80"
2005,"19,474,201.58"
2006,"17,637,704.07"
2007,"9,908,688.77"
2008,"11,909,857.55"
2009,"11,941,755.82"
2010,"18,454,638.72"
2011,"12,530,136.99"


In [51]:
metrics.opex("monthly")

,OpEx
month,
1,"10,197,423.93"
2,"10,677,887.48"
3,"9,812,309.86"
4,"12,868,926.27"
5,"10,916,038.51"
6,"19,944,128.71"
7,"15,549,668.82"
8,"5,499,528.09"
9,"10,052,144.07"


In [52]:
metrics.opex("month-year")

OpEx
year month             
2003 1       384,401.51
     2       356,490.34
     3       345,516.09
     4       323,658.55
     5       352,036.92
...                 ...
2012 8       579,766.09
     9       999,854.73
     10    4,544,805.64
     11    2,211,070.01
     12      357,432.76

[120 rows x 1 columns]

In [53]:
port_fees = pd.DataFrame([], columns=["port_fees"], index=metrics.labor_costs(frequency="month-year", by_type=False).index)
port_fees.fillna(0)

port_fees
year month           
2003 1              0
     2              0
     3              0
     4              0
     5              0
...               ...
2012 8              0
     9              0
     10             0
     11             0
     12             0

[120 rows x 1 columns]

<a id="process-times"></a>
## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [54]:
# Project totals at the project level
total = metrics.process_times()
total

,time_to_completion,process_time,downtime,N
category,,,,
annual service,"1,207,973.84","92,500.00","95,915.50",682
major repair,"63,086.83","1,850.40","1,906.00",26
major replacement,"231,098.87","9,630.88","231,098.87",55
manual reset,"351,358.44","39,396.25","37,614.75",5462
medium repair,"15,488.85","10,225.75","10,901.00",210
minor repair,"107,240.53","34,082.00","35,720.75",2246


<a id="power-production"></a>
## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [55]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

,windfarm
Project Energy Production (kWh),"9,433,597,783.50"


In [56]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

,windfarm,S00T1,S00T2,S00T3,S00T4,...,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"9,433,597,783.50","121,849,570.00","122,772,816.00","122,632,878.00","119,056,107.50",...,"117,986,021.50","114,164,379.00","121,080,442.00","115,768,130.50","122,048,930.00"


In [57]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"865,604,239.50"
2004,"886,368,986.50"
2005,"938,703,742.00"
2006,"932,716,703.00"
2007,"1,014,157,894.00"
2008,"1,043,102,659.00"
2009,"947,108,343.50"
2010,"841,360,562.00"
2011,"997,299,219.00"


In [58]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"1,058,205,404.00"
2,"796,624,760.50"
3,"870,579,706.00"
4,"684,494,188.50"
5,"666,799,885.50"
6,"540,288,030.50"
7,"578,008,845.50"
8,"630,110,544.50"
9,"777,561,859.00"


In [59]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False)

windfarm
year month               
2003 1     111,569,899.50
     2      58,872,091.50
     3      67,334,170.00
     4      92,308,746.50
     5      62,263,245.00
...                   ...
2012 8      52,942,387.00
     9      90,461,550.00
     10     89,397,036.00
     11     90,047,710.00
     12    105,992,030.50

[120 rows x 1 columns]

<a id="net-present-value"></a>
## Net Present Value

Here, we will go through the various input definitions to get the project NPV

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`discount_rate`: The rate of return that could be earned on alternative investments, by default 0.025. 

`offtake_price`: Price of energy, per MWh, by default 80.

In [60]:
metrics.npv("project")

,NPV
0,"197,555,821.22"


In [61]:
metrics.opex("annual")

,OpEx
year,
2003,"4,080,244.98"
2004,"16,315,800.80"
2005,"19,474,201.58"
2006,"17,637,704.07"
2007,"9,908,688.77"
2008,"11,909,857.55"
2009,"11,941,755.82"
2010,"18,454,638.72"
2011,"12,530,136.99"


In [62]:
metrics.opex("monthly")

,OpEx
month,
1,"10,197,423.93"
2,"10,677,887.48"
3,"9,812,309.86"
4,"12,868,926.27"
5,"10,916,038.51"
6,"19,944,128.71"
7,"15,549,668.82"
8,"5,499,528.09"
9,"10,052,144.07"


In [63]:
metrics.opex("month-year")

OpEx
year month             
2003 1       384,401.51
     2       356,490.34
     3       345,516.09
     4       323,658.55
     5       352,036.92
...                 ...
2012 8       579,766.09
     9       999,854.73
     10    4,544,805.64
     11    2,211,070.01
     12      357,432.76

[120 rows x 1 columns]

<a id="pysam-powered-results"></a>
## PySAM-Powered Results

For a number of project financial metrics, the PySAM library is utilized.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> If a "SAM_settings" file is not provided to the simulation, then the following metrics will not be able to be calculated and will raise a `NotImplementedError`.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [64]:
try:
    npv = metrics.pysam_npv()
    print(f"NPV: ${npv:,.0f}")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_npv()' cannot be calculated!


### Real Levelized Cost of Energy (LCOE)

In [65]:
try:
    lcoe = metrics.pysam_lcoe_real()
    print(f"Real LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_real()' cannot be calculated!


### Nominal Levelized Cost of Energy (LCOE)

In [66]:
try:
    lcoe = metrics.pysam_lcoe_nominal()
    print(f"Nominal LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_nominal()' cannot be calculated!


### After-tax Internal Return Rate (IRR)

In [67]:
try:
    npv = metrics.pysam_irr()
    print(f"IRR: {npv:,.1f}%")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_irr()' cannot be calculated!


### One Data Frame to Rule Them All

For this demonstration we will manually load a PySAM settings file and trigger the setup for demonstration purposes, but it should be noted that this practice should be avoided.

In [68]:
try:
    metrics.pysam_all_outputs()
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_all_outputs()' cannot be calculated!
